In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import scipy.stats as st
from scipy.stats import linregress

In [9]:
# Import API key
from api_keys import g_key

ImportError: cannot import name 'g_key' from 'api_keys' (/home/jupyter/python_api_challenge/WeatherPy/api_keys.py)

In [7]:
from citipy import citipy
# https://pypi.org/project/citipy/ (pip install citipy)
# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)